<a href="https://colab.research.google.com/github/tphlabs/Lab4_FH_teach/blob/main/FH_lab_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4. Frank-Hertz experiment

Igor Gitelman, 2024, v0.3

<p>Here we need to import measured data from the measuring program and then plot it. As you already know we need to plot the Frank-Hertz curve (characteristic curve).</p>

<p>We start by importing general modules that we use here, and in most of the lab experiments.</p>

In [3]:
import numpy as np # math functions
import matplotlib.pyplot as plt # for plotting figures and setting their properties
import pandas as pd # handling data structures (loaded from files)
from scipy.signal import find_peaks as find_peaks # Find peaks inside a signal based on peak properties.

<p> In the future we will use other modules too.</p>
<p>Now we use the <b>pandas</b> module, we called him <b>pd</b>, to import the data from file "FH1.csv", where the ending ".csv" stand for <b>comma separated value</b>. Pay attention to the actual separator, it may be different depending on the data type.</p>

In [17]:
# Place the path to your data here.
# When using Anaconda locally UPLOAD your data to the Anaconda environment.
URL = 'FH1.csv'

fh1=pd.read_csv(URL, sep='\t',header=5)
fh1.head()

,Va(V)_1,Ia(E-12 A)_1,T(c)_1,Va(V)_2,Ia(E-12 A)_2,T(c)_2
0,0.0,0.20,169.93,0.0,0.18,169.94
1,0.1,0.25,169.94,0.1,0.49,169.95
2,0.2,0.19,169.94,0.2,0.52,169.95
3,0.3,0.16,169.94,0.3,0.24,169.95
4,0.4,0.19,169.95,0.4,36.30,169.94


About the read_csv function, you can read [here](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) or you can google it.</p>
'\t' indicates a tab separator between columns.</p>
The function starts to read from line 5 that contains the column names, because of "header=5".

Now, take the names of the columns, and create relevant variables

In [11]:
Va1 = np.array(fh1['Va(V)_1']) # accelerating voltage vector
I1 = np.array(fh1['Ia(E-12 A)_1']) # Current vector
T1 = np.array(fh1['T(c)_1']) #temperature vector

Set the heater current, retard voltage depedning in your experiment parameters 

In [ ]:
Heater_Current=        #V
V_retard=              #V

one quick way to find the peaks of the characteristic curve is by using find_peaks function

In [12]:
peaks, _ = find_peaks(I1, prominence=10 ,width=3)
peaks

array([ 61, 108, 156, 205], dtype=int32)

peaks variable will contain the indexes in which the peaks occur

<h2>Data plot</h2>

<p>Now we use <b>matplotlib.pyplot</b> we called him <b>plt</b> in order to plot the data.</p>

In [33]:
plt.figure(dpi=200) # Define new figure, and set "Dots per Inch" or "dpi"|

<Figure size 1280x960 with 0 Axes>

<Figure size 1280x960 with 0 Axes>

And the distances between pick voltages

## Frank-Hertz runs at multiple temperatures/currents

To stay consistent with the rest of the lab we will read each CSV with `pandas.read_csv`, convert the voltage/current columns into `numpy` arrays, and then reuse `find_peaks` exactly as in the earlier example. Place the following files next to this notebook (tab separated with the same column names as `FH1.csv`, or edit the column names in the next cell if needed):

- `150c.csv`
- `160c.csv`
- `160c350mA.csv`
- `170c330mA.csv`
- `170c350mA.csv`
- `500mV350mA.csv`
- `5500mV350mA.csv`


In [ ]:
RUN_SPECS = {
    "150 °C": "150c.csv",
    "160 °C": "160c.csv",
    "160 °C / 350 mA": "160c350mA.csv",
    "170 °C / 330 mA": "170c330mA.csv",
    "170 °C / 350 mA": "170c350mA.csv",
    "500 mV / 350 mA": "500mV350mA.csv",
    "5500 mV / 350 mA": "5500mV350mA.csv",
}

data_tables = {}
for label, path in RUN_SPECS.items():
    try:
        df = pd.read_csv(path, sep='\t', header=5)
    except FileNotFoundError:
        print(f"Missing file: {path}")
        continue

    data_tables[label] = {"path": path, "data": df}
    print(f"{label} data preview ({path}):")
    display(df.head())


In [ ]:
def plot_run(voltage, current, label, prominence=8.0, width=3.0):
    peaks, _ = find_peaks(current, prominence=prominence, width=width)
    peak_voltages = voltage[peaks]

    plt.figure(dpi=200)
    plt.plot(voltage, current, label=label)
    plt.scatter(voltage[peaks], current[peaks], color='tab:red', zorder=3)

    for idx in peaks:
        plt.annotate(
            f"{voltage[idx]:.2f} V",
            (voltage[idx], current[idx]),
            xytext=(6, 8),
            textcoords='offset points',
            fontsize=9,
        )

    plt.title(f"Frank-Hertz curve @ {label}")
    plt.xlabel("Accelerating voltage (V)")
    plt.ylabel("Anode current (a.u.)")
    plt.grid(True, which='both', alpha=0.3)
    plt.legend()

    if len(peaks) == 0:
        print(f"{label}: no peaks found with the current parameters.")
    else:
        voltages_fmt = ', '.join(f'{v:.2f} V' for v in peak_voltages)
        print(f"{label}: peak voltages -> {voltages_fmt}")
    return peak_voltages

peak_rows = []
for label, payload in data_tables.items():
    df = payload["data"]
    path = payload["path"]

    try:
        voltage = np.array(df['Va(V)_1'])
        current = np.array(df['Ia(E-12 A)_1'])
    except KeyError as exc:
        print(f"{label}: missing column {exc}")
        continue

    peaks = plot_run(voltage, current, label)
    peak_rows.append(
        {
            "Run": label,
            "CSV": path,
            "Peak voltages (V)": ', '.join(f"{v:.2f}" for v in peaks) if len(peaks) else '—',
        }
    )

if peak_rows:
    summary_table = pd.DataFrame(peak_rows)
    display(summary_table)
else:
    print("No data files were loaded successfully.")


### Ionization analysis

The following cell repeats the same workflow for a single run but fits the ionization branch to an exponential current–voltage relation. The noisy low-voltage points are discarded before fitting so that both the linearized (log) plot and the I–V plot produce meaningful parameters.

In [ ]:
IONIZATION_RUN = "5500 mV / 350 mA"  # Change to the run you want to fit
NOISE_VOLTAGE_CUTOFF = 5.0  # Ignore voltages below this value (in V)
MIN_CURRENT = 0.01  # Ignore currents below this value (in the CSV units)

payload = data_tables.get(IONIZATION_RUN)
if payload is None:
    print(f"Run '{IONIZATION_RUN}' was not loaded. Check the file name above.")
else:
    df = payload["data"]
    try:
        voltage = np.array(df['Va(V)_1'], dtype=float)
        current = np.array(df['Ia(E-12 A)_1'], dtype=float)
    except KeyError as exc:
        print(f"Run '{IONIZATION_RUN}' is missing the column {exc}.")
    else:
        mask = (voltage >= NOISE_VOLTAGE_CUTOFF) & (current > MIN_CURRENT)
        if mask.sum() < 3:
            print("Not enough clean points after removing the noisy region. Adjust the cutoffs.")
        else:
            v_clean = voltage[mask]
            i_clean = current[mask]

            # Exponential fit: I = I0 * exp(k * V)
            slope, intercept = np.polyfit(v_clean, np.log(i_clean), 1)
            v_fit = np.linspace(v_clean.min(), v_clean.max(), 200)
            i_fit = np.exp(intercept) * np.exp(slope * v_fit)

            plt.figure(dpi=200)
            plt.plot(voltage, current, '.', color='gray', alpha=0.4, label='raw data')
            plt.plot(v_clean, i_clean, 'o', color='tab:blue', label='used for fit')
            plt.plot(v_fit, i_fit, '-', color='tab:red', label='exp fit')
            plt.title(f"Ionization I-V curve @ {IONIZATION_RUN}")
            plt.xlabel("Accelerating voltage (V)")
            plt.ylabel("Anode current (a.u.)")
            plt.grid(True, which='both', alpha=0.3)
            plt.legend()

            # Log(I) vs V plot (linearized)
            log_i_clean = np.log(i_clean)
            log_fit = slope * v_fit + intercept
            plt.figure(dpi=200)
            plt.scatter(v_clean, log_i_clean, color='tab:blue', label='log data (clean)')
            plt.plot(v_fit, log_fit, color='tab:red', label='linear fit')
            plt.title(f"ln(I) vs V @ {IONIZATION_RUN}")
            plt.xlabel("Accelerating voltage (V)")
            plt.ylabel("ln(I)")
            plt.grid(True, which='both', alpha=0.3)
            plt.legend()

            I0 = np.exp(intercept)
            print(f"Exponential fit parameters -> I(V) = {I0:.3e} * exp({slope:.3f} * V)")
            print(f"Used {mask.sum()} clean points out of {len(voltage)} total.")